# Introduction à xarray

`xarray` est une librarie Python conçue pour faciliter les manipulations de données sur grille. xarray peut lire différents formats de fichiers, dont netCDF et grib. Il existe beaucoup de matériel de formation sur xarray, et on ne présente ici qu'un très, très bref aperçu. 

Pour une présentation plus en profondeur, consultez http://xarray.pydata.org

In [ ]:
from __future__ import annotations

import xarray as xr

In [ ]:
%matplotlib inline

## Ouvrir un fichier

In [ ]:
ds = xr.open_dataset("../../../tests/testdata/CRCM5/tasmax_bby_198406_se.nc")
ds

dimension coordinates are one dimensional coordinates with a name equal to their sole dimension (marked by * when printing a dataset or data array).

## On accède aux dimensions par leur nom

Dans la plupart des interfaces netCDF, on manipule un cube de données en n-dimensions, et on doit se référer aux attributs pour savoir quel est l'index de la dimension time, lat lon. Si vous voulez accéder aux éléments du premier time step, il faut d'abord savoir si la dimension du temps et la première, deuxième ou troisième. 

On peut faire la même chose avec `xarray`:

In [ ]:
ds.tasmax[0]

Mais on peut aussi accéder aux données par le nom de leur dimension via leur index par la méhode `isel`: 

In [ ]:
ds.tasmax.isel(time=0)

où la valeur de la coordonnée par la méthode `sel`: 

In [ ]:
ds.tasmax.sel(time="1984-06-01")

## Faire un graphique ne demande pas tellement d'efforts

In [ ]:
ds.tasmax.sel(time="1984-06-01").plot(figsize=(10, 8), aspect="equal")

## On peut sélectionner plusieurs dimensions à la fois

Le méthode `sel` permet d'aller chercher les valeurs pour une coordonnée précise, une plage de coordonnées, ou même par plus proche voisin. Toutes ces sélections peuvent être mélangées dans n'importe quel ordre. 

In [ ]:
ds.tasmax.sel(time="1984-06-03", rlat=slice(-10, 15), rlon=slice(10, 35)).plot()

Ou aller chercher les valeurs de la coordonnées la plus proche:

In [ ]:
ds.tasmax.sel(rlat=34, method="nearest", tolerance=1).isel(time=0)

## Les opérations se font aussi selon les dimensions

Comme pour la sélection, les opérations sont effectuées le long de dimensions nommées explicitement. 

In [ ]:
# Le max pour tous les pas de temps
ds.tasmax.max(dim="time")

In [ ]:
# Le max pour chaque pas de temps
ds.tasmax.max(dim=("rlon", "rlat"))

## Les calendriers non-standards sont supportés

L'exemple plus haut utilise un calendrier `365_day`, automatiquement reconnu. 

## Le rééchantillonnage est trop facile 

On peut agréger une série à n'importe quelle échelle temporelle, peu importe que le calendrier soit standard ou pas. La méthode `resample` prend un paramètre de fréquence (`freq`) et retourne des groupes sur lesquels ont peut appliquer des opérateurs. 

In [ ]:
g = ds.tasmax.resample(time="3D")
g.groups

In [ ]:
g.mean()

In [ ]:
# g.mean().plot.line(marker='o')

## On peut faire facilement des vraies cartes projetées

In [ ]:
import cartopy.crs as ccrs
from matplotlib import pyplot as plt

# On crée la projection permettant d'interpréter les données brutes
rp = ccrs.RotatedPole(
    pole_longitude=ds.rotated_pole.grid_north_pole_longitude,
    pole_latitude=ds.rotated_pole.grid_north_pole_latitude,
    central_rotated_longitude=ds.rotated_pole.north_pole_grid_longitude,
)

# On crée la figure et ses axes. L'axe 1 utilise la projection native, et l'axe 2 une projection stéréographique.
fig = plt.figure(figsize=(18, 7))
ax1 = plt.subplot(121, projection=rp)
ax2 = plt.subplot(
    122, projection=ccrs.Stereographic(central_longitude=-100, central_latitude=45)
)

# On calcule la valeur à cartographier
x = ds.tasmax.mean(dim="time")

# Notez le paramètre "transform"
x.plot(ax=ax1, transform=rp, cbar_kwargs={"shrink": 0.8})
x.plot(ax=ax2, transform=rp, cbar_kwargs={"shrink": 0.8})
ax1.set_title("Rotated pole projection")
ax2.set_title("Stereographic projection")
ax1.coastlines()
ax2.coastlines()
plt.close()

In [ ]:
fig